## 数据加载

In [ ]:
# 数据加载
data= pd.read_csv('data/20171015/data_bank.csv',converters={'jrid':str})
##常用机器学习数据集加载 http://archive.ics.uci.edu/ml/index.php


### 数据预处理

In [ ]:
##### 脏数据处理
data = data.replace(['N',-1,'-1',-9999,'-9999'],np.nan)
data.replace(['N',-1,'-1',-9999,'-9999'],np.nan,inplace=True)

##### 缺失值处理
data.isnull()       #元素级别的判断，把对应的所有元素的位置都列出来，元素为空或者NA就显示True，否则就是False
data.isnull().any()　# 列级别的判断，只要该列有为空或者NA
missing=data.columns[data.isnull().any()].tolist() # 将为空或者NA的列找出来
data[missing].isnull().sum()  # 将列中为空或者NA的个数统计出来
##在kaggle中有人这样处理缺失数据，如果数据的缺失达到15%，且并没有发现该变量有多大作用，就删除该变量
data['column1']=data['column1'].fillna(0)  #.fillna(method='pad') .fillna(data['column1'].mean()) 
data.column1[data.column1.isnull()]=data.column1.dropna().mode().values #字符串类的数据剔除缺失值填充众数
data['column1'].interpolate() # 使用插值来估计NaN 如果index是数字，可以设置参数method='value' ，如果是时间，可以设置method='time'


##### 删除列
del data['column1']       #删除列
data['column1'].dropna()   #去掉为空值或者NA的元素
data.drop(['column1'],axis=1)     #去掉column1列，不管空值与否
df.drop(df.columns[[0,1]],axis=1,inplace=True)  #删除第1，2列，inplace=True表示直接就在内存中替换了，不用二次赋值生效。
data.dropna(axis=0)         #删除带有空值的行
data.dropna(axis=1)        #删除带有空值的列

##### 字符串替换
data['column1']=data['column1'].map({'RL':1,'RM':2,'RR':3,}).astype(int)


##### 时间数据处理
data['column1'] = (datetime.now() - pd.to_datetime(data['column1'].astype(str))).astype('timedelta64[D]')

#### 数据转换
data["column1"] = np.log1p(data["column1"])
data= pd.get_dummies(data) # 将字符串特征列中的内容提出来作为新的特征

#### 把内容为数值的特征列找出来
numeric_feats =data.dtypes[data.dtypes != "object"].index 


#### 将偏斜度大于0.75的数值列做一个log转换，使之尽量符合正态分布，因为很多模型的假设数据是服从正态分布的
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
data[skewed_feats] = np.log1p(data[skewed_feats])

#### 数据标准化
from sklearn import preprocessing
normalized_X = preprocessing.normalize(X)
standardized_X = preprocessing.scale(X)


### 数据描述

In [ ]:
# 统计分析
print("Skewness: %f" % data['column'].skew())   # 列出数据的偏斜度
print("Kurtosis: %f" % data['column'].kurt())　 # 列出数据的峰度
data['column1'].corr(data['column2'])           # 计算两个列的相关度
data['SqrtColumn1']=np.sqrt(data['column1'])    # 将列的数值求根，并赋予一个新列
data[['column1', 'column2']].groupby(['column2'], as_index=False).mean()  # f分组求均值


## 数据标准化

In [6]:
from sklearn import preprocessing
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
normalized_X = preprocessing.normalize(X)  # 归一化
standardized_X = preprocessing.scale(X)    # 标准化
rbscale_X = preprocessing.robust_scale(X)


## 特征选取

In [ ]:
# WOE特征离散化


## 算法调参

In [8]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(X, y)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.alpha)
#逻辑回归
from sklearn.linear_model import LogisticRegression  
X_matrix_train = data_loan_train_woe[data_loan_train_woe.columns[2:]].as_matrix(columns=None)
Y_matrix_train = data_loan_train_woe['overdue_m1'].as_matrix(columns=None)
tuned_parameters = [{'penalty':['l1','l2'],'C':[0.001,0.01,0.1,1,10,100]}]
clf = GridSearchCV(LogisticRegression(),tuned_parameters,cv=5,scoring='roc_auc')
clf.fit(X_matrix_train,Y_matrix_train)
clf.best_params_
lr = LogisticRegression(**clf.best_params_)  #最优模型参数get
lr_clf = lr.fit(X_matrix_train,Y_matrix_train)

##knn
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(p=2)
params = [{'penalty':['l1','l2'],'C':[0.001,0.01,0.1,10,100]}]
## DT
dt = tree.DecisionTreeClassifier(max_features=None,random_state=random)
params = [{'criterion':['gini','entropy'],'max_depth':list(range(4,8,1)),'min_samples_leaf':list(range(1,8,1))}]
## SVM 
svm = SVC(kernel='rbf',random_state=random,probability =True,class_weight='balanced')
params = [{'C':[0.001,0.01,10,100]}]
## RF
rf = RandomForestClassifier(n_estimators=100,random_state=random)
params = [{'n_estimators':[120,300],'max_depth':[5,8,15,None],'max_features':['sqrt','log2',None]}]
## AdaBoost
ab = AdaBoostClassifier(n_estimators=50,learning_rate=1.0,random_state=random)
params = [{'base_estimator':[DecisionTreeClassifier(),LogisticRegression()],'n_estimators':list(range(50,100,10))}]
## GBDT
GBDT = GradientBoostingClassifier(loss='deviance',random_state=random)
params = [{'n_estimators':[50,100],'learning_rate':[0.01,0.05,0.1],'subsample':[0.6,0.7,0.8,0.9,1.0],'max_depth':[5,8,15,25,30,None],
               'min_samples_leaf':[1,2,5,10],'max_features':['log2','sqrt',None]}]

## XGBoost
xgbc = xgb.XGBClassifier(objective= 'binary:logistic',scale_pos_weight=1,seed=random) 
params = [{'gamma':[0.05,0.1,0.3,0.5,0.7,0.9,1.0],'max_depth':[5,8,15,25,30,None],
           'min_samples_split':[2,5,10,15,100],'learning_rate':[0.01,0.015,0.025,0.05,0.1],
           'min_samples_leaf':[1,2,5,10],'max_features':['log2','sqrt',None]}]


### 模型评估

#### 1. 学习曲线
#####  偏差现象,损失函数值均较大，欠拟合
+ 增加假设模型的复杂度，欠拟合的本质是一些特征量在模型中未考虑。
+ 减少lambda，因为模型还未达到该有的复杂度，真正本质的原因就在于lambda就是表示bias偏量大小的
+ 在”偏差“现象中，通过增加训练样本的个数是无效的,是否加大训练样本的个数取决于是否有大的方差

##### 方差现象-损失函数值均较小，但两者之间始终有间隔，过拟合
+ 外在驱使，增加训练样本个数，提高了训练样本的复杂度
+ 内在驱动，降低特征值的个数（比如多项式拟合中的幂次等级）
+ 正规化处理，使用lambda权衡模型复杂度（Theta）

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

#学习曲线，了解模型的偏差方差信息
#偏差现象-损失函数值均较大，欠拟合
#方差现象-损失函数值均较小，但两者之间始终有间隔，过拟合


### 模型保存



In [ ]:
from sklearn.externals import joblib
joblib.dump(best_clf, file_dir+'result/all_model/tianji_{}.model'.format(title))

In [ ]:
###函数库
###计算KS
def ks_calc(data,score_col,class_col):
    '''
    功能: 计算KS值，输出对应分割点和累计分布函数曲线图
    输入值:
    data: 二维数组或dataframe，包括模型得分和真实的标签
    score_col: 一维数组或series，代表模型得分（一般为预测正类的概率）
    class_col: 一维数组或series，代表真实的标签（{0,1}或{-1,1}）
    输出值: 
    字典，键值关系为{'ks': KS值，'split': KS值对应节点，'fig': 累计分布函数曲线图}
    '''
    ks_dict = {}
    Bad = data.ix[data[class_col[0]]==1,score_col[0]]
    Good = data.ix[data[class_col[0]]==0, score_col[0]]
    ks,pvalue = stats.ks_2samp(Bad.values,Good.values)
    crossfreq = pd.crosstab(data[score_col[0]],data[class_col[0]])
    crossdens = crossfreq.cumsum(axis=0) / crossfreq.sum()
    crossdens['gap'] = abs(crossdens[0] - crossdens[1])
    score_split = crossdens[crossdens['gap'] == crossdens['gap'].max()].index[0]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    crossdens[[0,1]].plot(kind='line',ax=ax)
    ax.set_xlabel('%s' % score_col[0])
    ax.set_ylabel('Density')
    ax.set_title('CDF Curve of Classified %s' % score_col[0])
    plt.close()
    ks_dict['ks'] = ks
    ks_dict['split'] = score_split
    ks_dict['ks_fig'] = fig
    return ks_dict

###计算AUC
def auc_calc(data,score_col,class_col):
    '''
    功能: 计算AUC值，并输出ROC曲线
    输入值:
    data: 二维数组或dataframe，包括模型得分和真实的标签
    score_col: 一维数组或series，代表模型得分（一般为预测正类的概率）
    class_col: 一维数组或series，代表真实的标签（{0,1}或{-1,1}）
    输出值: 
    字典，键值关系为{'auc': AUC值，'auc_fig': ROC曲线}
    '''
    auc_dict = {}
#    fpr,tpr,threshold = roc_curve((1-data[class_col[0]]).ravel(),data[score_col[0]].ravel())   ###!!!!###
    fpr,tpr,threshold = roc_curve((data[class_col[0]]).ravel(),data[score_col[0]].ravel())
    roc_auc = auc(fpr,tpr)
    fig = plt.figure()
    plt.plot(fpr,tpr,color='b',label='ROC Curve (area=%0.3f)'%roc_auc,alpha=0.3)
    plt.plot([0,1],[0,1],color='r',linestyle='--',alpha=0.3)
    plt.xlim([0.0,1.0])
    plt.ylim([0.0,1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve of %s'%score_col[0])
    plt.legend(loc='lower right')
    plt.close()
    auc_dict['auc'] = roc_auc
    auc_dict['auc_fig'] = fig
    return auc_dict

###交叉验证计算模型测试集上平均AUC和KS
def fit_cv(X_matrix,Y_matrix,clf,cv,random):
    '''
    功能: 交叉验证计算模型测试集上平均AUC和KS
    输入值:
    X_matrix: 多维数组或dataframe，入模变量大宽表
    Y_matrix: 一维数组或series，真实的标签
    clf: 训练好的分类器
    cv: int，交叉验证中测试集的个数
    random: 随机排列的随机种子
    输出值: 
    字典，键值关系为{'auc': 测试集上平均AUC值，'ks': 测试集上平均KS值}
    '''
    skf = StratifiedKFold(y=Y_matrix,n_folds=cv,random_state=random)
    auc_list = []
    ks_list = []
    cv_dict = {}
    for train_index,test_index in skf:
        X_train,X_test = X_matrix[train_index],X_matrix[test_index]
        Y_train,Y_test = Y_matrix[train_index],Y_matrix[test_index]
        fit_data = pd.DataFrame(index=range(len(test_index)),columns=['Y','Prob'])
        clf = clf.fit(X_train,Y_train)
        fit_data['Y'] = Y_test
        fit_data['Prob'] = clf.predict_proba(X_test)[:,1]
        score_col = ['Prob']
        class_col = ['Y']
        auc_ = auc_calc(fit_data,score_col,class_col)['auc']
        auc_list.append(auc_)
        ks_ = ks_calc(fit_data,score_col,class_col)['ks']
        ks_list.append(ks_)
        cv_dict['ks'] = np.mean(ks_list)
        cv_dict['auc'] = np.mean(auc_list)
    return cv_dict

###画好坏人分数分布对比直方图
def plot_hist_score(y_true,y_score,close=True):
    '''
    功能: 画好坏人分数分布对比直方图
    y_true: 一维数组或series，代表真实的标签（{0,1}或{-1,1}）
    y_score: 一维数组或series，代表模型得分
    close: 是否关闭图片
    返回图片对象
    '''
    good = y_score[y_true==1]
    bad = y_score[y_true==0]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.hist(bad,bins=100,alpha=0.6,color='r',label='Bad')
    ax.hist(good,bins=100,alpha=0.6,color='b',label='Good')
    ax.set_xlabel('Score')
    ax.set_ylabel('Frequency')
    ax.legend(loc='best')
    ax.set_title('Histogram of Score in Good vs Bad')
    if close:
        plt.close('all')
    return fig

###学习曲线
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    
    fig = plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,train_scores_mean + train_scores_std, alpha=0.1,color="r")
    
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,test_scores_mean + test_scores_std, alpha=0.1, color="g")
    
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation score")

    plt.legend(loc="best")
    plt.close()
    fig.savefig(file_dir+'{}.png'.format(title))
    
    return plt